In [14]:
import numpy as np
from itertools import permutations
import copy
import time

import gymnasium as gym
from gymnasium import spaces
from gridworld.envs import GridWorldEnv, SiblingGridWorldEnv
from gymnasium.wrappers import FlattenObservation
from gridworld.wrappers import RelativePosition, InvertedReward
from gymnasium.envs.registration import register

from utils import *

In [4]:
def sibling():
    env = SiblingGridWorldEnv(P_gridworld)
#     env = RelativePositionenv)
    return env

register(
    id='SiblingGridWorld-v0',
    entry_point=sibling,
    max_episode_steps=300,
)

env = gym.make('SiblingGridWorld-v0')
env = env.unwrapped
env.render_mode = 'human'
obs, info = env.reset()

In [25]:
rng = np.random.default_rng()
for i in range(300):
    Q, V, pi = value_iteration(env.cur_P)
    obs, rew, done, trunc, info = env.step([pi(5*obs[1]+obs[0]), rng.integers(0, 24, size=1, dtype=int)[0]])
    time.sleep(0.05)
    if done:
        print(f"Finished in {i} steps.")
        break

Finished in 42 steps.


In [15]:
env.close()